In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [2]:
BH_tweets_before_pandemic = pd.read_csv('dane/BH_tweets_before_pandemic.csv', lineterminator='\n')

In [3]:
BH_tweets_before_pandemic.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,8.078300e+04,8.078300e+04,80783.0,8.078300e+04,80783.000000,80783.000000,80783.000000,80783.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,9.842332e+17,9.841421e+17,200.0,1.912607e+17,0.187552,0.906391,2.427627,0.266455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.314246e+16,3.337859e+16,0.0,3.589090e+17,4.847892,18.562753,114.526056,0.442107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,9.142809e+17,4.171556e+17,200.0,3.240000e+02,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,9.584344e+17,9.584322e+17,200.0,9.066943e+07,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,9.842697e+17,9.841305e+17,200.0,7.017780e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.010695e+18,1.010555e+18,200.0,3.828431e+09,0.000000,0.000000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.046548e+18,1.046548e+18,200.0,1.044284e+18,1092.000000,2350.000000,30265.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_before_pandemic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80783 entries, 0 to 80782
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               80783 non-null  int64  
 1   conversation_id  80783 non-null  int64  
 2   created_at       80783 non-null  object 
 3   date             80783 non-null  object 
 4   time             80783 non-null  object 
 5   timezone         80783 non-null  int64  
 6   user_id          80783 non-null  int64  
 7   username         80783 non-null  object 
 8   name             80780 non-null  object 
 9   place            638 non-null    object 
 10  tweet            80783 non-null  object 
 11  language         80783 non-null  object 
 12  mentions         80783 non-null  object 
 13  urls             80783 non-null  object 
 14  photos           80783 non-null  object 
 15  replies_count    80783 non-null  int64  
 16  retweets_count   80783 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_before_pandemic.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1035675683450761217,1035675683450761217,2018-09-01 01:48:19 CEST,2018-09-01,01:48:19,200,27569109,albeesgirl,Staci Rullo,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1035673787356864512,1035673787356864512,2018-09-01 01:40:47 CEST,2018-09-01,01:40:47,200,3296463285,k_val81,Karen Valentine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1035672623026397184,1035670992448675841,2018-09-01 01:36:09 CEST,2018-09-01,01:36:09,200,2322821784,stevet_140,Steve T,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'NastromKenneth', 'name': 'Ke...",NaN,NaN,NaN,NaN
3,1035669895151738880,1035669895151738880,2018-09-01 01:25:19 CEST,2018-09-01,01:25:19,200,332364947,simonsalhanyre,BHHS Simon & Salhany,"{'type': 'Point', 'coordinates': [41.193133, -...",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1035668062610116609,1035668062610116609,2018-09-01 01:18:02 CEST,2018-09-01,01:18:02,200,918188324426194944,stand_4_america,Free America Network,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format and sorted by dates
BH_tweets_before_pandemic['date'] = pd.to_datetime(BH_tweets_before_pandemic['date'], format = '%Y-%m-%d')
BH_tweets_before_pandemic = BH_tweets_before_pandemic.sort_values(by='date')
BH_tweets_before_pandemic = BH_tweets_before_pandemic.reset_index(drop=True)
BH_tweets_before_pandemic

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,914280914749927424,914280914749927424,2017-10-01 02:08:51 CEST,2017-10-01,02:08:51,200,564818181,atlanticitynj,AtlantiCityNJDirect,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,914589886191108096,914589886191108096,2017-10-01 22:36:35 CEST,2017-10-01,22:36:35,200,849762128,bhhspremier,Berkshire Hathaway HomeServices Premier Proper...,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,914584278092189697,914584278092189697,2017-10-01 22:14:18 CEST,2017-10-01,22:14:18,200,4852197763,ntkrnow,kbc,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,914582734181404674,914582734181404674,2017-10-01 22:08:10 CEST,2017-10-01,22:08:10,200,565612962,dpolitical,Daily Political,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,914580596080480257,914580596080480257,2017-10-01 21:59:40 CEST,2017-10-01,21:59:40,200,2266388972,midetimes,MidEast Times,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80778,1046545458112462849,1046545458112462849,2018-10-01 01:40:55 CEST,2018-10-01,01:40:55,200,335046054,loveshomesinnj,Tʜᴇ Bᴀʀᴅᴀᴄʜ Tᴇᴀᴍ NJ Rᴇᴀʟᴛᴏʀs,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
80779,1046540474784210944,1046540474784210944,2018-10-01 01:21:07 CEST,2018-10-01,01:21:07,200,2503230883,_real__estate,Real Estate News,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
80780,1046533765638606848,1046533765638606848,2018-10-01 00:54:28 CEST,2018-10-01,00:54:28,200,24117274,srodes,Susie Rodes,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
80781,1046531227417759745,1046531227417759745,2018-10-01 00:44:22 CEST,2018-10-01,00:44:22,200,376898714,andreaszlavikte,Andrea Szlavik Team,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
#Remove tweets writeen in other language than english
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic[BH_tweets_before_pandemic.language == 'en']
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.reset_index(drop=True)
BH_tweets_before_pandemic_clean.shape

(76149, 36)

In [8]:
#Remove columns without any data
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.dropna(thresh = 1, axis = 1)

In [9]:
#2360 records and 10 columns were removed
BH_tweets_before_pandemic_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,7.614900e+04,7.614900e+04,76149.0,7.614900e+04,76149.000000,76149.000000,76149.000000,76149.000000
mean,9.843070e+17,9.842162e+17,200.0,1.921056e+17,0.190154,0.919342,2.475686,0.267633
std,3.320385e+16,3.344878e+16,0.0,3.595004e+17,4.990037,19.056316,117.942848,0.442728
min,9.142809e+17,4.171556e+17,200.0,3.240000e+02,0.000000,0.000000,0.000000,0.000000
25%,9.584318e+17,9.584298e+17,200.0,8.812009e+07,0.000000,0.000000,0.000000,0.000000
50%,9.843252e+17,9.841534e+17,200.0,7.050270e+08,0.000000,0.000000,0.000000,0.000000
75%,1.010911e+18,1.010855e+18,200.0,3.843177e+09,0.000000,0.000000,0.000000,1.000000
max,1.046548e+18,1.046548e+18,200.0,1.044284e+18,1092.000000,2350.000000,30265.000000,1.000000


In [10]:
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76149 entries, 0 to 76148
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               76149 non-null  int64         
 1   conversation_id  76149 non-null  int64         
 2   created_at       76149 non-null  object        
 3   date             76149 non-null  datetime64[ns]
 4   time             76149 non-null  object        
 5   timezone         76149 non-null  int64         
 6   user_id          76149 non-null  int64         
 7   username         76149 non-null  object        
 8   name             76146 non-null  object        
 9   place            624 non-null    object        
 10  tweet            76149 non-null  object        
 11  language         76149 non-null  object        
 12  mentions         76149 non-null  object        
 13  urls             76149 non-null  object        
 14  photos           76149 non-null  objec

In [11]:
BH_tweets_before_pandemic_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,914280914749927424,914280914749927424,2017-10-01 02:08:51 CEST,2017-10-01,02:08:51,200,564818181,atlanticitynj,AtlantiCityNJDirect,NaN,...,0,0,"['business', 'member']",[],https://twitter.com/AtlantiCityNJ/status/91428...,False,NaN,1,https://pbs.twimg.com/media/DLAtpTtWsAEA5qM.jpg,[]
1,914589886191108096,914589886191108096,2017-10-01 22:36:35 CEST,2017-10-01,22:36:35,200,849762128,bhhspremier,Berkshire Hathaway HomeServices Premier Proper...,NaN,...,0,0,[],[],https://twitter.com/bhhspremier/status/9145898...,False,NaN,0,NaN,[]
2,914584278092189697,914584278092189697,2017-10-01 22:14:18 CEST,2017-10-01,22:14:18,200,4852197763,ntkrnow,kbc,NaN,...,0,0,[],[],https://twitter.com/NTKRNow/status/91458427809...,False,NaN,0,NaN,[]
3,914582734181404674,914582734181404674,2017-10-01 22:08:10 CEST,2017-10-01,22:08:10,200,565612962,dpolitical,Daily Political,NaN,...,0,0,"['brk', 'y']","['brk', 'y']",https://twitter.com/DPolitical/status/91458273...,False,NaN,0,NaN,[]
4,914580596080480257,914580596080480257,2017-10-01 21:59:40 CEST,2017-10-01,21:59:40,200,2266388972,midetimes,MidEast Times,NaN,...,0,0,[],"['brk', 'y']",https://twitter.com/midetimes/status/914580596...,False,NaN,0,NaN,[]


In [12]:
#Check values of column place
BH_tweets_before_pandemic_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [33.9032486, -84.46694957]}     32
{'type': 'Point', 'coordinates': [36.067886, -115.272926]}       14
{'type': 'Point', 'coordinates': [40.7399954, -74.0301392]}      12
{'type': 'Point', 'coordinates': [30.32487, -81.39882]}          10
{'type': 'Point', 'coordinates': [32.2125, -80.8973]}             9
                                                                 ..
{'type': 'Point', 'coordinates': [41.221649, -81.617074]}         1
{'type': 'Point', 'coordinates': [40.0, -80.0]}                   1
{'type': 'Point', 'coordinates': [40.7153285, -73.9446043]}       1
{'type': 'Point', 'coordinates': [40.8223, -74.212]}              1
{'type': 'Point', 'coordinates': [29.42657568, -98.43972092]}     1
Name: place, Length: 418, dtype: int64

In [13]:
#Check values of column quote_url
BH_tweets_before_pandemic_clean['quote_url'].value_counts()

0                                                                     67
https://twitter.com/FoxBusiness/status/1034087310084722690             8
https://twitter.com/coindesk/status/990969761080250371                 7
https://twitter.com/nytimes/status/958322214260047872                  7
https://twitter.com/BTCTN/status/964305989100072965                    5
                                                                      ..
https://twitter.com/QTRResearch/status/993440375678398465              1
https://twitter.com/craigcalcaterra/status/992716066546597888?s=21     1
https://twitter.com/globeandmail/status/1034392939277869056            1
https://twitter.com/cullenroche/status/1034199840362717185             1
https://twitter.com/CNBCnow/status/963884503516278787                  1
Name: quote_url, Length: 939, dtype: int64

In [14]:
#Counted quantity of links for each tweet
BH_tweets_before_pandemic_clean['quote_url'] = BH_tweets_before_pandemic_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_before_pandemic_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_before_pandemic_clean['quote_url'] = BH_tweets_before_pandemic_clean['quote_url'].astype(np.int64)

In [15]:
#Replayce NaN to [] in thumbnail column
BH_tweets_before_pandemic_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [16]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76149 entries, 0 to 76148
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               76149 non-null  int64         
 1   conversation_id  76149 non-null  int64         
 2   created_at       76149 non-null  object        
 3   date             76149 non-null  datetime64[ns]
 4   time             76149 non-null  object        
 5   timezone         76149 non-null  int64         
 6   user_id          76149 non-null  int64         
 7   username         76149 non-null  object        
 8   name             76146 non-null  object        
 9   place            624 non-null    object        
 10  tweet            76149 non-null  object        
 11  language         76149 non-null  object        
 12  mentions         76149 non-null  object        
 13  urls             76149 non-null  object        
 14  photos           76149 non-null  objec

In [17]:
#Check duplicates
BH_tweets_before_pandemic_clean[BH_tweets_before_pandemic_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [18]:
#Column retweet consist of only False values
BH_tweets_before_pandemic_clean['retweet'].value_counts()

False    76149
Name: retweet, dtype: int64

In [19]:
#Removed irrelevant columns
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.drop(['conversation_id', 
                                                                        'created_at', 'time', 'timezone', 
                                                                        'language', 'retweet', 'link','place'], 
                                                                       axis = 1)

In [20]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_before_pandemic_clean[column] == '[]'
    BH_tweets_before_pandemic_clean[column] = BH_tweets_before_pandemic_clean[column].str[1:-1].str.split(',')
    BH_tweets_before_pandemic_clean[column] = BH_tweets_before_pandemic_clean[column].str.len()
    BH_tweets_before_pandemic_clean.loc[empty_list, column] = 0

In [21]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_before_pandemic_clean['reply_to'] = BH_tweets_before_pandemic_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_before_pandemic_clean['reply_to'] = BH_tweets_before_pandemic_clean['reply_to'].astype(np.int64)

In [22]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_before_pandemic_clean.shape[0]):
    BH_tweets_before_pandemic_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_before_pandemic_clean.loc[i, 'tweet']).split())

In [23]:
#Last day 2018-10-01 has only 10 tweets, it will be removed
BH_tweets_before_pandemic_clean['date'].value_counts()

2018-01-30    8050
2018-01-31    2528
2018-05-05    2032
2018-05-04    1691
2018-02-24    1384
              ... 
2017-10-28      40
2018-01-01      40
2017-10-01      31
2017-10-08      29
2018-10-01      10
Name: date, Length: 366, dtype: int64

In [24]:
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean[BH_tweets_before_pandemic_clean['date'] != '2018-10-01']
BH_tweets_before_pandemic_clean['date'].value_counts()

2018-01-30    8050
2018-01-31    2528
2018-05-05    2032
2018-05-04    1691
2018-02-24    1384
              ... 
2018-01-01      40
2017-10-28      40
2017-10-15      40
2017-10-01      31
2017-10-08      29
Name: date, Length: 365, dtype: int64

In [25]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_before_pandemic_clean[~BH_tweets_before_pandemic_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [26]:
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76139 entries, 0 to 76138
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              76139 non-null  int64         
 1   date            76139 non-null  datetime64[ns]
 2   user_id         76139 non-null  int64         
 3   username        76139 non-null  object        
 4   name            76136 non-null  object        
 5   tweet           76139 non-null  object        
 6   mentions        76139 non-null  int64         
 7   urls            76139 non-null  int64         
 8   photos          76139 non-null  int64         
 9   replies_count   76139 non-null  int64         
 10  retweets_count  76139 non-null  int64         
 11  likes_count     76139 non-null  int64         
 12  hashtags        76139 non-null  int64         
 13  cashtags        76139 non-null  int64         
 14  quote_url       76139 non-null  int64         
 15  vi

In [27]:
#Save cleaned data in csv format
BH_tweets_before_pandemic_clean.to_csv('dane/BH_tweets_before_pandemic_clean.csv', index=False)

In [30]:
#Create dataframe for textanalysis for each day
BH_tweets_before_pandemic_textanalysis = pd.date_range(BH_tweets_before_pandemic_clean['date'].min(),
                                                       BH_tweets_before_pandemic_clean['date'].max(),freq='d')
BH_tweets_before_pandemic_textanalysis = pd.DataFrame(BH_tweets_before_pandemic_textanalysis, columns = ['date'])
column_names = ['Negative factual tweets', 'Negative personal tweets', 'Positive factual tweets',
                'Positive personal tweets', 'Neutral factual tweets', 'Neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_before_pandemic_textanalysis[column] = 0
BH_tweets_before_pandemic_textanalysis

,date,Negative factual tweets,Negative personal tweets,Positive factual tweets,Positive personal tweets,Neutral factual tweets,Neutral personal tweets,Total amount of tweets
0,2017-10-01,0,0,0,0,0,0,0
1,2017-10-02,0,0,0,0,0,0,0
2,2017-10-03,0,0,0,0,0,0,0
3,2017-10-04,0,0,0,0,0,0,0
4,2017-10-05,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
360,2018-09-26,0,0,0,0,0,0,0
361,2018-09-27,0,0,0,0,0,0,0
362,2018-09-28,0,0,0,0,0,0,0
363,2018-09-29,0,0,0,0,0,0,0


In [31]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_before_pandemic_clean.shape[0]):
    blob = TextBlob(BH_tweets_before_pandemic_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if blob.sentiment[0] < 0:
        if blob.sentiment[1] < 0.5:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Negative factual tweets'] += 1           
        else:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Negative personal tweets'] += 1
    elif blob.sentiment[0] > 0:
        if blob.sentiment[1] < 0.5:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Positive factual tweets'] += 1
        else:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Positive personal tweets'] += 1
    else:
        if blob.sentiment[1] < 0.5:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Neutral factual tweets'] += 1
        else:
            BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                            'Neutral personal tweets'] += 1
    BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [32]:
BH_tweets_before_pandemic_textanalysis

,date,Negative factual tweets,Negative personal tweets,Positive factual tweets,Positive personal tweets,Neutral factual tweets,Neutral personal tweets,Total amount of tweets
0,2017-10-01,1,0,4,2,20,4,31
1,2017-10-02,3,0,11,3,37,1,55
2,2017-10-03,29,1,113,36,451,1,631
3,2017-10-04,1,2,33,29,108,0,173
4,2017-10-05,2,1,16,11,66,1,97
...,...,...,...,...,...,...,...,...
360,2018-09-26,40,4,56,94,56,3,253
361,2018-09-27,17,3,81,55,83,6,245
362,2018-09-28,12,3,45,51,70,5,186
363,2018-09-29,6,2,27,28,37,2,102


In [33]:
BH_tweets_before_pandemic_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      365 non-null    datetime64[ns]
 1   Negative factual tweets   365 non-null    int64         
 2   Negative personal tweets  365 non-null    int64         
 3   Positive factual tweets   365 non-null    int64         
 4   Positive personal tweets  365 non-null    int64         
 5   Neutral factual tweets    365 non-null    int64         
 6   Neutral personal tweets   365 non-null    int64         
 7   Total amount of tweets    365 non-null    int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 22.9 KB


In [34]:
#Save text analysis data in csv format
BH_tweets_before_pandemic_textanalysis.to_csv('dane/BH_tweets_before_pandemic_textanalysis.csv', index=False)